In [1]:
from build_Case1 import build_Case1
import numpy as np
from scipy.interpolate import interp1d
import pickle
import pandas as pd
import uuid
import time
from pathlib import Path
import io

In [2]:
rootPath = '/nobackup/jakravit/data/'
runPath = rootPath + 'HE60/run/'
batch_name = 'case1V1'
clearIOPprofiles = True
numRuns = 200

if clearIOPprofiles:
    # clear iop profiles
    %rm -rf /nobackup/jakravit/data/HE60/iop_profiles/{batch_name}
    %mkdir /nobackup/jakravit/data/HE60/iop_profiles/{batch_name}
    # clear batch files
    %rm -rf /nobackup/jakravit/data/HE60/run/batch
    %mkdir /nobackup/jakravit/data/HE60/run/batch
    # clear runlist
    # %rm /nobackup/jakravit/data/HE60/run/runlist.txt  
    # Path('/nobackup/jakravit/data/HE60/run/runlist.txt').touch()
    # clear outputs
    %rm -rf /nobackup/jakravit/data/HE60/output/EcoLight/excel
    %mkdir /nobackup/jakravit/data/HE60/output/EcoLight/excel   
    # clear batchruns file
    %rm /nobackup/jakravit/data/HE60/run/batchruns.sh 
    with io.open(runPath + 'batchruns.sh', 'a+') as f:
        f.write('#!/bin/bash\n') 
    # clear pickles
    %rm -rf /nobackup/jakravit/data/HE60/pickles
    %mkdir /nobackup/jakravit/data/HE60/pickles 

rm: cannot remove '/nobackup/jakravit/data/HE60/run/batchruns.sh': No such file or directory


In [3]:
# Data paths 

# phytoplankton SIOP spectral library
phyPath = rootPath+'EAP_phytoplankton_dataset/PFTs.p'
with open(phyPath, 'rb') as fp:
    phy_library = pickle.load(fp)

# NAP spectral libraries
minpath = rootPath+'EAP_NAP_dataset/minerals_V2.p'
with open(minpath, 'rb') as fp:
    datamin = pickle.load(fp)  

detpath = rootPath+'EAP_NAP_dataset/det_V1.p'
with open(detpath, 'rb') as fp:
    datadet = pickle.load(fp) 

# Benthic library
benthic_lib = pd.read_csv(rootPath+'benthic_spec_libary_FT_V4.csv')

# adjacency library
adj_lib = pd.read_csv(rootPath+'adjacency_spectra_V2.csv')

# aeronet library
aero_lib = pd.read_csv(rootPath+'aeronet_invdata_match.csv')

In [4]:
rows = []
uidList = []
for k in range(numRuns):
    iops, cols, row = build_Case1(phy_library, datamin, datadet, benthic_lib, adj_lib, aero_lib)
    rows.append(row)

    #% TOTAL IOPS

    # depth
    depth = iops['Depth']['Depth']
    depth = depth.reshape(-1, 1)
    xfactor = iops['Depth']['xfactor']
    dlen = len(depth)

    # chl profile
    chlProfile = [iops['Phyto']['TotChl'] * x for x in xfactor]
    chlProfile = np.asarray(chlProfile).reshape(-1, 1)
    foot = np.hstack((-1,0))
    chlzProfile = np.hstack((depth,chlProfile))
    chlzProfile = np.vstack((chlzProfile,foot))

    # absorption
    atotphy = iops['Phyto']['a_tot']
    atotmin = iops['Min']['a_tot']
    atotdet = iops['Det']['a_tot']
    atotdom = iops['CDOM']['a_tot'][63:]
    atot = atotphy + atotmin + atotdet + atotdom
    a_tot = [atot * x for x in xfactor]
    a_totphy = [atotphy * x for x in xfactor]
    a_totmin = [atotmin * x for x in xfactor]
    a_totdet = [atotdet * x for x in xfactor]
    a_totdom = [atotdom * x for x in xfactor]

    # Scatter
    btotphy = iops['Phyto']['b_tot']
    btotmin = iops['Min']['b_tot']
    btotdet = iops['Det']['b_tot']
    btot = btotphy + btotmin + btotdet
    # b_tot = [btot * x for x in xfactor]

    # attenuation
    ctotphy = atotphy + btotphy
    ctotmin = atotmin + btotmin
    ctotdet = atotdet + btotdet
    ctotdom = atotdom # no scattering
    ctot = atot + btot
    c_tot = [ctot * x for x in xfactor]
    c_totphy = [ctotphy * x for x in xfactor]
    c_totmin = [ctotmin * x for x in xfactor]
    c_totdet = [ctotdet * x for x in xfactor]
    c_totdom = [ctotdom * x for x in xfactor]

    # backscattering
    bbtotphy = iops['Phyto']['bb_tot']
    bbtotmin = iops['Min']['bb_tot']
    bbtotdet = iops['Det']['bb_tot']
    bbtot = bbtotphy + bbtotmin + bbtotdet
    bb_tot = [bbtot * x for x in xfactor]
    bb_totphy = [bbtotphy * x for x in xfactor]
    bb_totmin = [bbtotmin * x for x in xfactor]
    bb_totdet = [bbtotdet * x for x in xfactor]
    
    #% PREPARE FOR WRITING TO .TXT FILE IN HYDROLIGHT FORMAT

    # lambda
    l = np.arange(400,902.5,2.5)

    llen = len(l)
    line11 = np.hstack((llen,l)).reshape(1,-1)
    # AC data
    foot = np.hstack((-1,[0]*(llen*2)))
    # only phy
    acProfile_phy = np.hstack((depth,a_totphy,c_totphy))
    acProfile_phy = np.vstack((acProfile_phy,foot))
    # only min
    acProfile_min = np.hstack((depth,a_totmin,c_totmin))
    acProfile_min = np.vstack((acProfile_min,foot))
    # only det
    acProfile_det = np.hstack((depth,a_totdet,c_totdet))
    acProfile_det = np.vstack((acProfile_det,foot))
    # only dom
    acProfile_dom = np.hstack((depth,a_totdom,c_totdom))
    acProfile_dom = np.vstack((acProfile_dom,foot))
    # all components
    acProfile = np.hstack((depth,a_tot,c_tot))
    acProfile = np.vstack((acProfile,foot))  

    # bb data
    foot = np.hstack((-1,[0]*(l)))
    # only phy
    bbProfile_phy = np.hstack((depth,bb_totphy))
    bbProfile_phy = np.vstack((bbProfile_phy,foot))
    # only min
    bbProfile_min = np.hstack((depth,bb_totmin))
    bbProfile_min = np.vstack((bbProfile_min,foot))
    # only det
    bbProfile_det = np.hstack((depth,bb_totdet))
    bbProfile_det = np.vstack((bbProfile_det,foot))
    # all components
    bbProfile = np.hstack((depth,bb_tot))
    bbProfile = np.vstack((bbProfile,foot))

    #% WRITE TO TXT
    
    import string
    import random
    import os 

    iopPath = rootPath+'HE60/iop_profiles/{}/'.format(batch_name)
    if not os.path.exists(iopPath):
        os.mkdir(iopPath)
        
    def random_uid():
        alphabet = string.ascii_lowercase + string.digits
        return ''.join(random.choices(alphabet, k=10))
    uid = random_uid()
    uidList.append(uid)

    # sname.append(uid)
    iopzpath = iopPath+'{}/'.format(uid)
    if not os.path.exists(iopzpath):
        os.mkdir(iopzpath)
    # os.mkdir(rootPath + 'HE60/iop_profiles/{}/{}/'.format(batch_name,uid))
    # iopzpath = rootPath + 'HE60/iop_profiles/{}/{}/'.format(batch_name,uid)

    # acData

    # fnames
    fnames = {'total': {'acfname': 'acDatatot_' + uid + '.txt',
                        'bbfname': 'bbDatatot_' + uid + '.txt',
                        'acdata': acProfile,
                        'bbdata': bbProfile},
              'phyto': {'acfname': 'acDataphy_' + uid + '.txt',
                        'bbfname': 'bbDataphy_' + uid + '.txt',
                        'acdata': acProfile_phy,
                        'bbdata': bbProfile_phy},
              'min':   {'acfname': 'acDatamin_' + uid + '.txt',
                        'bbfname': 'bbDatamin_' + uid + '.txt',
                        'acdata': acProfile_min,
                       'bbdata': bbProfile_min},
              'det':   {'acfname': 'acDatadet_' + uid + '.txt',
                        'bbfname': 'bbDatadet_' + uid + '.txt',
                        'acdata': acProfile_det,
                        'bbdata': bbProfile_det},
              }

    # write chl data
    header = '\n'.join(['CHL CONC DATA PROFILE',
                        'SWIPE HE data files',
                        '#',
                        '#','#','#','#','#','#','#\n'])
    with open(os.path.join(iopzpath,'chlzData_{}.txt'.format(uid)),'w') as f:
        f.writelines(header)
        np.savetxt(f,chlzProfile,delimiter='\t')

    # write AC data
    header = '\n'.join(['TOTAL AC DATA PROFILE',
                        'SWIPE HE data files',
                        'Does not include water!',
                        '#','#','#','#','#','#','#\n'])
    for fname in fnames:
        with open(os.path.join(iopzpath,fnames[fname]['acfname']),'w') as f:
            f.writelines(header)
            np.savetxt(f,line11,fmt='%d',delimiter='\t')
            np.savetxt(f,fnames[fname]['acdata'],delimiter='\t')

    # write bb data
    header = '\n'.join(['TOTAL BB DATA PROFILE',
                        'SWIPE HE data files',
                        'Does not include water!',
                        '#','#','#','#','#','#','#\n'])
    for fname in fnames:
        with open(os.path.join(iopzpath,fnames[fname]['bbfname']),'w') as f:
            f.writelines(header)
            np.savetxt(f,line11,fmt='%d',delimiter='\t')
            np.savetxt(f,fnames[fname]['bbdata'],delimiter='\t')

    # write iroot file        
    from write_iroot import iroot
    iroot(l, iops, batch_name, uid, rootPath)
    
    # write pickle file
    # with open(rootPath + 'HE60/pickles/{}.p'.format(uid), 'wb') as f:
    #     pickle.dump(iops,f)
            
sims = pd.DataFrame(rows,columns=cols, index=uidList)
sims.to_csv(rootPath + 'HE60/batch_data/{}_table.csv'.format(batch_name))
print ('DONE!')

DONE!


In [5]:
iops['Phyto'].keys()

dict_keys(['TotChl', 'Haptophytes', 'Dinoflagellates', 'Raphidophyte', 'Heterokonts', 'Eustigmatophyte', 'Cryptophytes', 'Diatoms', 'Cyano_red', 'Rhodophytes', 'a_tot', 'b_tot', 'c_tot', 'bb_tot'])

In [6]:
sims.head()

,TotChl,ph_a_tot_400.0,ph_a_tot_402.5,ph_a_tot_405.0,ph_a_tot_407.5,ph_a_tot_410.0,ph_a_tot_412.5,ph_a_tot_415.0,ph_a_tot_417.5,ph_a_tot_420.0,...,atm_Std-F,atm_VolC-C,atm_REff-C,atm_VMR-C,atm_Std-C,prof,OZA,OAA,SZA,SAA
ujdz8v5err,0.388,0.012600,0.012871,0.013040,0.013273,0.013469,0.013669,0.013820,0.013943,0.014000,...,0.508,0.174,1.849,2.204,0.580,afglt,15,90,45,30
j933nx9buu,0.438,0.017855,0.018353,0.018842,0.019408,0.019963,0.020505,0.020974,0.021351,0.021588,...,0.393,0.022,2.693,3.340,0.649,afglss,15,100,65,60
7fqb0vaabi,0.316,0.007498,0.007683,0.007787,0.007915,0.008018,0.008123,0.008202,0.008265,0.008290,...,0.446,0.064,2.002,2.513,0.677,afglss,15,80,45,30
9z5dc7dxpk,0.406,0.020332,0.020849,0.021308,0.021932,0.022456,0.022926,0.023273,0.023609,0.023907,...,0.466,0.071,2.023,2.554,0.679,afglsw,40,85,50,40
mtv4q55ifd,0.417,0.012572,0.012990,0.013170,0.013445,0.013638,0.013842,0.013974,0.014093,0.014143,...,0.706,0.395,1.797,2.114,0.563,afglmw,30,70,15,50


In [7]:
iops.keys()

dict_keys(['VSF_angles', 'lambda', 'Phyto', 'Min', 'Det', 'CDOM', 'Depth', 'Benthic', 'Adjacency', 'Atm'])